**Feature Extraction Settings:**
- New DS project - start by extracting a comprehensive set of features. then maybe later you can identify which features are relevant for the task at hand. In the final stages - then maybe finetune the parameter of the features to fine tune your model. 
- Can do all those things with tsfresh. What you need to know is - how to control which features are calculated by tsfresh and how can one adjsut the parameters.

Need to know how to control which features are calculated by tsfresh and how one can adjust the parameters of the features.

**For the lazy: Just calculate some features**
- To calculate a comprehensive set of features - call the tsfresh.extract_features() method without passing a default_fc_parameters or kind_to_fc_parameters object. this way you'll be using the default options which will use all the feature calculators in this package. 

**For the advanced: How to set the parameters for all kind of time series?**
- After digging deep into your data - you maybe want to calculate more of a certain type of feature and less of another type. So - need to use custom settings for feature extractors. To do that with tsfresh - you have to use a custom settings object:

In [ ]:
from tsfresh.feature_extraction import ComprehensiveFCParameters, extract_features

settings = ComprehensiveFCParameters()
# set the options of the settings object as shown in the paragraphs below.

extract_features(df, default_fc_parameters=settings)

The default_fc_parameters is expected to be a dictionary which maps feature calculator names (the function names you can find in the tsfresh.feature_extraction.feature_calculators file) to a list of dictionaries - which are the parameters with which the function will be called as key value pairs. Each function-parameter combination that is in this dictionary will be called during extraction and will produce a feature. If the function does not take any parameters, the value should be set to None.

In [ ]:
fc_parameters = {
    "length": None, 
    "large_standard_deviation": [{"r": 0.05}, {"r": 0.1}]
}

The above code will produce 3 features: one by calling the tsfresh.feature_extraction.feature_calculators.length() function without any parameters and two by calling tsfresh.feature_extraction.feature_calculators.large_standard_deviation() with r = 0.05 and r = 0.1.

- So, we can control which features will be extracted, by adding or removing either keys or parameters from this dictionary. If you decide not to calculate the length feature here, you can just delete it from the dictionary:

In [ ]:
del fc_parameters['length']

And now - only two features are calculated.

In [2]:
from tsfresh.feature_extraction.settings import ComprehensiveFCParameters

For convenience, three dictionaries are predefined and can be used right away:
- tsfresh.feature_extraction.settings.ComprehensiveFCParameters: includes all features without parameters and all features with parameters, each with different parameter combinations. This is the default for tsfresh.extract_features() - if you do not hand in a default_fc_parameters at all. 
- tsfresh.feature_extraction.settings.MinimalFCParameters: includes only a handful of features and can be used for quick tests. The features which have the minimal attribute are used here
- tsfresh.feature_extraction.settings.EfficientFCParameters: Mostly the same features as in ComprehensiveFCParameters, but without features which are marked with the "high_comp_cost" attribute. Can be used if runtime performance plays a major role.

Theoretically - you could calculate an unlimited number of features with tsfresh by adding entry adter entry to the dictionary.

**For the ambitious: How do I set the parameters for different type of time series?**
- It is also possible to control the features to be extracted for the different kinds of time series individually. Can do so by passing a kind_to_fc_parameters to the tsfresh.extract_features() function. It should be a dict mapping from kind names (as strings) to fc_parameters ibjects, which you'd normally pass as arguments to the default_fc_parameters parameter.

In [ ]:
kind_to_fc_parameters = {
    "temperature": {"mean": None},
    "pressure": {"maximum": None, "minimum": None}
}

The above will extract the "mean" of the "temperature" time series and the "minimum" and "maximum" of the "pressure" time series. 
- The kind_to_fc_parameters argument will partly override the default_fc_parameters. So, if you include a kind name in the kind_to_fc_parameters parameter, its value will be used for that kid. Other kinds will still still use the default_fc_parameters.

**Handy Trick: Do I really have to create the dictionary by hand?**
- Not necessarily. Assuming you have a DataFrame of tsfresh features. By using feature selection algorithms you find out only a subset is relevant. 

- Then there is the tsfresh.feature_extraction.settings.from_columns() method that constructs the kind_to_fc_parameters dictionary from the column names of this filtered feature matrix to make sure that only relevant features are extracted. 

- This can save a huge amount of time because you prevent the calculation of unndecessary features

In [ ]:
# X_tsfresh contains the extracted tsfresh features
X_tsfresh = extract_features()

# which are bow filtered to only contain relevant features
X_tsfresh_filtered = some_feature_selection(X_tsfresh, y,...)

# we can easily construct the corresponding settings object
kind_to_fc_parameters = tsfresh.feature_extraction.settings.from_columns(X_tsfresh_filtered)

Above code snippet will construct the kind_to_fc_parameters dictionary that corresponds to the features and parameters from the tsfresh features that were filtered by the some_feature_selection feature selection algo